## .csv to .npy

In [62]:
data = pd.read_csv("data.csv")
dict1 = dict(zip(data['name'],data['score']))
np.save('kiaa_bmt_data.npy', dict1)

## main

In [65]:
def lim(x, uplim=20):
    '''将得分限制在0～uplim'''
    if x > uplim: return uplim
    if x < 0:  return 0
    return x

def get_score_1(p1_score, p2_score, p1_res, p2_res, print=False):
    '''
    # 用于计算KIAA羽毛球俱乐部积分赛的积分（单打）
    # 分数范围：0～100
    # rank = 分数 / 10
    # Parameters
        p1_score: 比赛前P1的分数
        p2_score: 比赛前P2的分数
        p1_res  : 对局中P1的得分
        p2_res  : 对局中P2的得分
    # Returns:
        p1_score_new: 比赛后P1的分数
        p2_score_new: 比赛后P2的分数
    '''
    
    def K(score):
        '''放大系数'''
        return 2 - 0.01 * score
    
    # 超过 20 分计为 20
    p1_res = lim(p1_res)
    p2_res = lim(p2_res)
    
    p1_exp = lim(20 + (p1_score - p2_score) / 10 * 4)  # P1的预期得分
    p2_exp = lim(20 - (p1_score - p2_score) / 10 * 4)  # P2的预期得分
    
    # 实际得分和预期的差
    dif = (p1_res - p1_exp) - (p2_res - p2_exp)
    if print:
        print("预期比分: {:.1f} : {:.1f}".format(p1_exp, p2_exp))
        print("实际比分: {:.1f} : {:.1f}".format(p1_res, p2_res))
    
    p1_score_new = lim(p1_score + K(p1_score) * dif, 150)
    p2_score_new = lim(p2_score - K(p2_score) * dif, 150)
    if print:
        print("赛后双方的分数变化：")
        print("P1: {:.1f} --> {:.1f}\nP2: {:.1f} --> {:.1f}".format(p1_score, p1_score_new,
                                                                p2_score, p2_score_new))
    return p1_score_new, p2_score_new


def get_score_2(t1_p1_score, t1_p2_score, t2_p1_score, t2_p2_score, t1_res, t2_res):
    '''
    # 用于计算双打积分
    # Parameters
        t1(2)_p1(2)_score: 比赛前Team1(2)中P1(2)的分数
        t1(2)_res  : 对局中Team1(2)的得分
    # Returns:
        t1(2)_p1(2)_score_new: 比赛后Team1(2)中P1(2)的分数
    '''
    t1 = (t1_p1_score + t1_p2_score) / 2
    t2 = (t2_p1_score + t2_p2_score) / 2
    t1_new, t2_new = get_score_1(t1, t2, t1_res, t2_res)
    
    t1_p1_score_new = lim( t1_p1_score + (t1_new - t1) / 2, 150)
    t1_p2_score_new = lim( t1_p2_score + (t1_new - t1) / 2, 150)
    t2_p1_score_new = lim( t2_p1_score + (t2_new - t2) / 2, 150)
    t2_p2_score_new = lim( t2_p2_score + (t2_new - t2) / 2, 150)
    
    return t1_p1_score_new, t1_p2_score_new, t2_p1_score_new, t2_p2_score_new

# 读取以字典形式存储的数据
data = np.load("kiaa_bmt_data.npy", allow_pickle=True).item()

def update_data(t1_p1, t1_p2, res1, res2, t2_p1=None, t2_p2=None):
    '''
    # 用于更新数据库中的积分
    # Parameters
        t1(2)_p1(2): Team1(2)中P1(2)的姓名拼音（单打只有Team1）
        res1(2)    : 对局比分
    '''
    if t2_p1 is None:
        data[t1_p1], data[t1_p2] = get_score_1(data[t1_p1], data[t1_p2], res1, res2)
    else:
        data[t1_p1], data[t1_p2], data[t2_p1], data[t2_p2] = \
        get_score_2(data[t1_p1], data[t1_p2], data[t2_p1], data[t2_p2], res1, res2)
    #np.save("kiaa_bmt_data.npy", data)

In [66]:
# 比赛
update_data("WangShun", "FuShuqi", 21,  8, "LiHulin", "LinXiaojing")
update_data("WangShun", "FuShuqi", 21, 13, "LiHulin", "LinXiaojing")
update_data("XuFengwei", "YuNiankun", 21, 14, "LiuChengqi", "ZouSiwei")
update_data("XuFengwei", "YuNiankun", 21, 19, "LiuChengqi", "ZouSiwei")
update_data("LyuCheqiu", "LiHulin", 6, 21)
update_data("LyuCheqiu", "LiHulin", 8, 21)
update_data("YuNiankun", "ZuoPei", 18, 21, "WangHuimei", "LinXiaojing")
update_data("YuNiankun", "ZuoPei", 14, 21, "WangHuimei", "LinXiaojing")
update_data("XuFengwei", "ZhangLulu", 18, 21, "HuangQifeng", "SunShengxiu")
update_data("XuFengwei", "ZhangLulu", 21, 12, "HuangQifeng", "SunShengxiu")
update_data("XuFengwei", "ZhangLulu", 21, 17, "HuangQifeng", "SunShengxiu")
update_data("XueZihan", "Alex", 14, 21, "ShiYangfan", "XuFuxiang")
update_data("XueZihan", "Alex", 10, 21, "ShiYangfan", "XuFuxiang")
update_data("ZhuRui", "ChenSiyuan", 24, 22, "XuFuxiang", "YinGaocheng")
update_data("ZhuRui", "ChenSiyuan", 8, 21, "XuFuxiang", "YinGaocheng")
update_data("ZhuRui", "ChenSiyuan", 8, 21, "XuFuxiang", "YinGaocheng")

# 5/9
update_data("Alex", "ZhangLulu",9,21)
update_data("PanZhiwei", "YinGaocheng", 21, 15, "XuFuxiang", "SunShengxiu")
update_data("XuFengwei", "FuShuqi", 24, 22, "WangShun", "WangHuimei")

sorted(data.items(), key=lambda x:x[1], reverse=1)

[('PangYuxuan', 150),
 ('WangShun', 144.12277193895957),
 ('PanZhiwei', 138.29564343549657),
 ('LiuChengqi', 121.17552),
 ('XuFengwei', 118.54079530803452),
 ('LiHulin', 117.06650547190928),
 ('FeiQinyue', 100),
 ('LyuCheqiu', 85.77508606747385),
 ('ShiYangfan', 83.0355),
 ('WangHuimei', 80.57534753352499),
 ('FuShuqi', 74.81434308169852),
 ('SunShengxiu', 69.42075585605177),
 ('HuangQifeng', 67.42906697204378),
 ('XueZihan', 67.29650000000001),
 ('YuNiankun', 65.20745177507071),
 ('XuFuxiang', 60.516588308791114),
 ('ZhangLulu', 56.98612117544153),
 ('LinXiaojing', 45.78669559456542),
 ('ZouSiwei', 41.17552),
 ('ZuoPei', 36.24189177507071),
 ('ZhuRui', 24.453197704402598),
 ('YinGaocheng', 23.7850428602797),
 ('Alex', 23.78207539705064),
 ('YueNannan', 10),
 ('ChenSiyuan', 4.453197704402598)]